### Creating a fake case-control data set for neonatal metabolomics studies 
We often work with case control studies where the goal is to identify factors (metabolites) that may be associated/contribute to a certain medical condition. This is done by comparing patient/subjects who have the condition/disease (the "cases") with patients who do not have the condition/disease but are otherwise similar (the "controls"). 

In studies involving dry-blood spots sampled from newborns (as part of the national newborn screening program), controls are selected to have the same (or very close in time) date of birth as the cases. This is because the metabolomic profiles of newborns show a strong variation with season [***reference***?], possibly reflecting how the mother's  (and the rest of us) diet/ lifetyle varies depedning on the time of year it is. 


#### The code
The code below will create a fake data set having the typical structure of a record with sample subjects paired into case and control sample types. We create a pandas dataframe with this data to visualize the resulting table and allow easy export to a csv and/or excel file. 

For all random data we use the [`Faker` library](https://faker.readthedocs.io/en/master/index.html), with the [faker-biology plugin](https://pypi.org/project/faker-biology/) available for Python. 

##### Parameters
We create `n_pairs` of case and control pairs, all having a random date of birth within a defined minimum and maximum age. The maximum age difference between each pair can be defined; below we allow a maximum age difference of seven days, but months and years can also be chosen. 

We also define additional sub groups by assigning a random organ from a list of  `n_organ_groups`; this could perhaps be the tissue from were a certain condition is being monitored in the study. A random barcode is assigned to each subject, which could perhaps be an identifer used to identfy the sample in a biobank say. 






### Setup libs

In [13]:
import pandas as pd
import numpy as np

import faker
from faker_biology.physiology import CellType, Organ, Organelle

from dateutil.relativedelta import relativedelta


# init fake object and load plugins
fake = faker.Faker()

# add organ data plugin 
fake.add_provider(Organ)

# or organelle and celtype plugin
# fake.add_provider(Organelle)
# fake.add_provider(CellType)

### Set parameters

In [14]:
# save
save_to_excel = True

# number of case control pairs in fake study 
n_pairs = 523 

# age of subjects
min_age = 15 # years
max_age = 40 # years

# max age difference between case and control
max_diff_years = 0
max_diff_months = 0
max_diff_days = 7

# number of sub groups; here organs to simulate some attribute of the disease/condition
n_groups = 5
organs = [fake.organ() for _ in range(0,n_groups)]

variables = ["pair_ID", "specimen_ID", "object", "date_of_birth", "year", "barcode", "organ"]

# create dict to hold the fake data 
fake_data = {}
for v in variables:
    fake_data.setdefault(v, [])

In [15]:
organs

['Tendons', 'Human skeleton', 'Lymphatic vessel', 'Ciliary body', 'Ileum']

### Create fake data

In [16]:
for pair_id in range(0, n_pairs):
    
    # Choose a random sub group
    case_control_organ = np.random.choice(organs)
    
    # --- Controls ---
    
    dob_control = fake.date_of_birth(minimum_age=min_age, maximum_age=max_age)# datetime object
    spec_id_control = np.random.randint(1,n_pairs*2+1)
    
    fake_data["pair_ID"].append(pair_id)
    fake_data["object"].append("Control")
    fake_data["specimen_ID"].append(spec_id_control)
    fake_data["date_of_birth"].append(dob_control)
    fake_data["year"].append(dob_control.year)
    fake_data["barcode"].append(fake.ean8())
    fake_data["organ"].append(case_control_organ)
    
    # --- Cases ---
    
    # let the case specimen be born within relativedelta time from control
    dob_case = fake.date_between_dates(dob_control, 
                            dob_control + relativedelta(years=max_diff_years, 
                                          month=max_diff_months, 
                                          days=max_diff_days)
                            )
    
    # let case and control specimen numbers be contiguous
    spec_id_case = spec_id_control + 1
    
    fake_data["pair_ID"].append(pair_id)
    fake_data["object"].append("Case")
    fake_data["specimen_ID"].append(spec_id_case)
    fake_data["date_of_birth"].append(dob_case)
    fake_data["year"].append(dob_case.year)
    fake_data["barcode"].append(fake.ean8())
    fake_data["organ"].append(case_control_organ)

In [19]:
fake_data_df = pd.DataFrame(fake_data)
fake_data_df.sort_values(by="year", inplace=True)

if save_to_excel:
    filename = f"fake_case_control_Npairs_{n_pairs}_Ngroups_{n_groups}.xlsx"
    fake_data_df.to_excel("data/" + filename)
    
if save_to_excel:
    filename = f"fake_case_control_Npairs_{n_pairs}_Ngroups_{n_groups}.csv"
    fake_data_df.to_csv("data/" + filename)

In [20]:
fake_data_df

,pair_ID,specimen_ID,object,date_of_birth,year,barcode,organ
796,398,467,Control,1982-09-30,1982,53516701,Tendons
844,422,466,Control,1982-07-26,1982,38591754,Human skeleton
843,421,353,Case,1982-06-25,1982,31105699,Ileum
842,421,352,Control,1982-06-20,1982,15086068,Ileum
123,61,334,Case,1982-11-07,1982,79433747,Ileum
...,...,...,...,...,...,...,...
755,377,165,Case,2007-12-16,2007,87131338,Tendons
159,79,324,Case,2008-02-11,2008,22513892,Human skeleton
83,41,550,Case,2008-02-25,2008,16069152,Ileum
82,41,549,Control,2008-02-25,2008,60179333,Ileum
